# Setup

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D,InputLayer,Flatten,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical 
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

## getting data

In [ ]:
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
sns.countplot(df_train.label)

In [ ]:
X = (df_train.iloc[:,1:].values/255)
y = df_train.iloc[:,:1]

In [ ]:
plt.figure(  figsize = (10,12) )
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X.reshape(-1,28,28,1)[i],cmap = 'gray')
    plt.title(("Value: {}".format(y.iloc[i,0])))
plt.show()

# train - test split

In [ ]:
y = to_categorical(y, num_classes = 10)
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.25, random_state = 42)
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

# model 

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,activation = 'relu',padding = 'Same' ,input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size = (2,2), strides = 2))
model.add(BatchNormalization())
model.add(Conv2D(32,3,activation = 'relu',padding = 'Same' ,input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size = (2,2), strides = 2))
model.add(BatchNormalization())
model.add(Conv2D(32,3,activation = 'relu', padding = 'Same'))
model.add(MaxPool2D(pool_size = (2,2), strides = 2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))

model.add(Dense(10, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy', metrics=["accuracy"])

In [ ]:
early_stopping = EarlyStopping(patience=40, monitor = 'val_loss')

In [ ]:
history = model.fit( x_train, y_train,validation_data=(x_valid,y_valid),  epochs = 100, callbacks = [early_stopping],batch_size = 100)

In [ ]:
model.evaluate(x_valid, y_valid)

In [ ]:
plt.plot(history.history['loss'], label = 'train_loss')
plt.plot(history.history['val_loss'], label = 'valid_loss')
plt.ylim(0,0.20)
plt.title('loss functions')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label = 'train_accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.ylim(0.980,1)
plt.title('accuracy functions')
plt.show()

# looking results

In [ ]:
results = model.predict(x_valid).argmax(axis=1)
plt.figure(  figsize = (10,12) )
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_valid[i])
    plt.title(("Value: {}".format(results[i])))
plt.show()

# last

In [ ]:
x_test = df_test.iloc[:].values/255
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
prediction = pd.DataFrame({'ImageId':range(1,28001),'Label': model.predict(x_test).argmax(axis=1)})
prediction

In [ ]:
prediction.info()

In [ ]:
prediction.to_csv('my_submission',index = None)